In [2]:
import pandas as pd

# Read input files
products_df = pd.read_csv('/content/products.csv')
sales_df = pd.read_csv('/content/sales.csv')

# Merge products with sales on SKU
merged_df = pd.merge(products_df, sales_df, on='sku', how='left')
merged_df['quantity_sold'] = merged_df['quantity_sold'].fillna(0)

def apply_pricing_rules(row):
    current_price = row['current_price']
    cost_price = row['cost_price']
    stock = row['stock']
    quantity_sold = row['quantity_sold']
    new_price = current_price

    if stock < 20 and quantity_sold > 30:
        new_price = current_price * 1.15
    elif stock > 200 and quantity_sold == 0:
        new_price = current_price * 0.7
    elif stock > 100 and quantity_sold < 20:
        new_price = current_price * 0.9

    min_price = cost_price * 1.2
    if new_price < min_price:
        new_price = min_price
        return round(current_price, 2), round(new_price, 2)

merged_df[['old_price', 'new_price']] = merged_df.apply(
    apply_pricing_rules, axis=1, result_type='expand'
)


merged_df['old_price'] = merged_df['old_price'].apply(lambda x: f"${x}")
merged_df['new_price'] = merged_df['new_price'].apply(lambda x: f"${x}")


output_df = merged_df[['sku', 'old_price', 'new_price']]
output_df.to_csv('updated_prices.csv', index=False)
